# This is for STM32F un-masked aes implementation for STM32F3

In [ ]:
SCOPETYPE='OPENADC'
PLATFORM='CW308_STM32F3'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_1_1'

board = 'stm32f3'

# This is for xmega un-masked aes implementation for XMEGA

In [ ]:

SCOPETYPE='OPENADC'
PLATFORM='CWLITEXMEGA'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_1_1'

board = 'xmega'

In [ ]:
%run jupyter/Setup_Scripts/Setup_Generic.ipynb

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 

In [ ]:
cmd = "hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, cmd)
print(cmd)

# Prameters Offset and Number of Samples per datasets:

In [ ]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
key, text = ktp.next()
target.set_key(key)

scope.adc.offset = 0
scope.adc.samples = 5000
ktp = cw.ktp.Basic()

# Key Setup

In [ ]:
dir(ktp)
aaa = ktp.getInitialKey()
#print(aaa)
bbb = ktp.getKeyType()
print(bbb)

# The appropriate key can be uncommented based on the sspecific dataset,
# note that the default key is automatically set : We reffer to the default key as K1
#
# Here are the Different Key options that we use in data collection: 


# K2
# key_str = 'aa,80,d8,a7,84,d3,3f,5c,0b,90,a9,85,20,8e,ff,4a'

# K3
# key_str = 'd2,d5,01,68,82,83,91,43,96,9e,e9,a2,53,a7,52,e1'

# K4
# key_str = 'e6,de,35,a9,a5,23,19,df,c6,cc,bb,ba,c1,36,c3,bf'


# ktp.fixed_key = False
# ktp.setInitialKey(key_str)
ktp.fixed_key = True
key, text = ktp.next()
print(key)

# SAMPLE KEY AND TXT BOX

In [ ]:
target.set_key(key)
for i in range(3):
    key, text = ktp.next()
    print(key)
    print(text)
    print('==============')

# 10 Sample Traces: 

In [ ]:
trace_array = []
textin_array = []
N = 10
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
from matplotlib.pyplot import MultipleLocator

x_locator = MultipleLocator(2000)
ax = plt.gca()
ax.xaxis.set_major_locator(x_locator)

plt.plot(trace_array[2][0:5000], color='r')

# SCOPE PARAMETERS: 

In [ ]:
scope

# DATA COLLECTION:

In [ ]:
trace_array = []
textin_array = []
N = 300000
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 

# Scope Specs

In [ ]:
print("Capture rate: "+str
      (scope.clock.adc_freq/10e5)+"MHz")

# Save the data

In [ ]:
output_path='/home/ucsec/Desktop/'
os.makedirs(output_path, exist_ok=True)

file_name='val_diff_key.npz'

outpath = os.path.join(output_path,file_name)
np.savez(outpath, power_trace=trace_array, plain_text=textin_array,key=key)


In [ ]:
hex_key = [hex(val) for val in key]
print(hex_key)

In [ ]:
scope.dis()
target.dis()